## MaskデータをCOCOフォーマットに変換して学習用のデータセットを作成する
[参考:COCO Formatの作り方](https://qiita.com/harmegiddo/items/da131ae5bcddbbbde41f)

## フォーマット
データ全体のフォーマットは以下の通り  
今回必要そうなのは、  
- Info
- Images
- Annotations
```JSON
{
    "info": {...},
    "licenses": [...],
    "images": [...],
    "annotations": [...],
    "categories": [...], <-- Not in Captions annotations
    "segment_info": [...] <-- Only in Panoptic annotations
}

```

## Info部分の作成
あってもなくてもいいがデータセットの内容について記述するために用意する

### フォーマット
```JSON
"info": {
    "description": "COCO 2017 Dataset",
    "url": "http://cocodataset.org",
    "version": "1.0",
    "year": 2017,
    "contributor": "COCO Consortium",
    "date_created": "2017/09/01"
}
```

In [1]:
# info部分の作成
import json
import collections as cl

def info():
    tmp = cl.OrderedDict() # OrderedDict:値を追加した順序を記憶することができる辞書型風のデータ構造を提供
    tmp["descripion"] = "Predicting_Solar_Flare"
    tmp["version"] = "0.1"
    tmp["year"] = 2020
    tmp["contributor"] = "Akito Komatsu"
    tmp["data_created"] = "2020/09/01"
    return tmp
def main():
    query_list = ["info","images","annotations"]
    js = cl.OrderedDict()
    for i in range (len(query_list)):
        tmp = ""
        if query_list[i] == "info":
            tmp =info()
        
        js[query_list[i]] = tmp
    fw = open("datasets.json","w")
    json.dump(js,fw,indent=2)

if __name__ == "__main__":
    main()



In [2]:
dataset_obj = open("datasets.json","r")
dataset_json =json.load(dataset_obj)
dataset_json

{'info': {'descripion': 'Predicting_Solar_Flare',
  'version': '0.1',
  'year': 2020,
  'contributor': 'Akito Komatsu',
  'data_created': '2020/09/01'},
 'images': '',
 'annotations': ''}

## imagesの作成
IDはユニークで後のAnnotation等に関連付けられる  
→time？検討
フォーマット
```JSON
"images": [
    {
        "license": 4,
        "file_name": "000000397133.jpg",
        "coco_url": "http://images.cocodataset.org/val2017/000000397133.jpg",
        "height": 427,
        "width": 640,
        "date_captured": "2013-11-14 17:02:52",
        "flickr_url": "http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg",
        "id": 397133
    },
    {
        "license": 1,
        "file_name": "000000037777.jpg",
        "coco_url": "http://images.cocodataset.org/val2017/000000037777.jpg",
        "height": 230,
        "width": 352,
        "date_captured": "2013-11-14 20:55:31",
        "flickr_url": "http://farm9.staticflickr.com/8429/7839199426_f6d48aa585_z.jpg",
        "id": 37777
    },
    ...
]
```

## 画像データの作成
そもそもCOCOフォーマットのデータを作成する前に画像データ自体を作成しなければならないので指定のフォーマットのデータを作成できるようにする流れは 
- [ ] FITSファイルから画像データの書き出し
- [ ] FITSファイルのメタデータから必要なものを抜きだす


In [33]:
# そもそもの画像の方のデータセットも同時に作る
from glob import glob
import sunpy.map
import cv2
fits_path = "/media/akito/Data/HMI_REGION/2010/*2010050*"
paths = sorted(glob(fits_path))
for i, path in enumerate(paths):
    filename="/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/"+path.split(".")[2][0:15]+".jpg"
    filename="/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/"+str(i).zfill(3)+".jpg"
    map=sunpy.map.Map(path)
    print(filename)
    cv2.imwrite(filename,cv2.flip(map.data,1))




/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/000.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/001.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/002.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/003.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/004.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/005.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/006.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/007.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/008.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/009.jpg
/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Da

## 画像のフォーマットについて
Numpyを書き出したら白黒の4k4kのデータになっているがこれで大丈夫そうか？
